In [1]:
import pandas_helper
import lendingclub_helper
import scikit_helper
import sklearn as sk
import numpy as np
import pandas as pd

In [2]:
train_data_file = "finished_train_sample.csv"
df = pandas_helper.readData(train_data_file, lendingclub_helper.dtypes)
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,1.000000e+03,0.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.000000,21.000000,21.000000
mean,3.997987e+07,NaN,14764.550000,14748.300000,14699.039029,452.296850,77917.934470,17.614940,0.300000,696.560000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5361.642857,47.245714,13.285714
std,3.253072e+07,NaN,8566.801453,8570.828867,8584.553123,265.259847,59750.445198,8.276416,0.843037,32.272402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2803.349960,5.822807,8.538986
min,1.798970e+05,NaN,1000.000000,1000.000000,0.000000,33.210000,10000.000000,0.030000,0.000000,660.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,891.000000,30.000000,0.000000
25%,8.999260e+06,NaN,8000.000000,8000.000000,8000.000000,254.675000,46000.000000,11.580000,0.000000,670.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3505.000000,45.000000,6.000000
50%,3.540877e+07,NaN,13875.000000,13875.000000,13812.500000,389.460000,65000.000000,16.900000,0.000000,690.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5931.230000,45.000000,18.000000
75%,6.636608e+07,NaN,20000.000000,20000.000000,20000.000000,592.882500,93000.000000,22.917500,0.000000,711.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7537.720000,50.000000,18.000000
max,1.226793e+08,NaN,40000.000000,40000.000000,40000.000000,1347.570000,980000.000000,45.160000,12.000000,845.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9842.000000,60.000000,24.000000


In [24]:
lendingclub_helper.buildDateFeature(df, "hardship_start_date")

In [25]:
feature_columns = lendingclub_helper.getFeatureColumns(df)
feature_columns

Index(['f_hardship_start_date_diff', 'f_hardship_start_date_month_NaT',
       'f_hardship_start_date_month_NULL'],
      dtype='object')

In [22]:
col = "hardship_start_date"
month_col = col + "_month"  # create temporary column to parse month name
df[month_col] = pd.to_datetime(df[col]).dt.strftime('%b')
# df[month_col] = pd.to_datetime(df[col]).dt.strftime('%b').str.replace('NaT', "NULL")
lendingclub_helper.buildOneHotEncodingFeature(df, month_col)
# df.drop(month_col, axis=1, inplace=True)  # drop temporary column



In [20]:
df.drop("f_hardship_start_date_month_NULL", axis=1, inplace=True)

In [50]:
v = df[col][0]
pd.to_datetime(v)
# pd.to_datetime(df[col]).dt.strftime('%b')


NaT

In [28]:
df[col].isnull().all()


True

In [22]:
for c in feature_columns:
    try:
        sk.utils.assert_all_finite(df[c].values)
    except Exception as e:
        print(e)
        print(c)
        break

Input contains NaN, infinity or a value too large for dtype('float64').
f_hardship_start_date_diff


In [29]:
# df[["hardship_start_date", "f_hardship_start_date_month_NULL", "f_hardship_start_date_diff"]]
hardship_columns = [c for c in df.columns if "hardship_start_date" in c]
print(hardship_columns)

['hardship_start_date', 'f_hardship_start_date_month_Dec', 'f_hardship_start_date_month_NaT', 'f_hardship_start_date_month_Oct', 'f_hardship_start_date_diff']
